In [1]:
import cvxpy as cp
import numpy as np

import models.helper as helper

def solve_rsdr_with_selected_antennas(H, gamma, sigma_sq, epsi, z_mask, z_sol):
    """
    off_ants: antenna indices that need to be turned off
    """

    N, M = H.shape
    H_short = H.copy()

    print(z_mask, z_sol)
    for n in range(N-1, -1, -1):
        print(z_sol[n], z_mask[n])
        if z_mask[n] and not z_sol[n]:
            print('pruning', n)
            H_short = np.concatenate((H_short[:n, :], H_short[n+1:, :]), axis=0)
    print(N,M)
    print('h short', H_short.shape)

    num_off_ants = np.sum(z_mask*(1-z_sol))
    assert num_off_ants<N, 'number of allowed antennas < 1'
    N = N - num_off_ants    
    
    W = []
    for i in range(M):
        W.append(cp.Variable((N,N), hermitian=True))

    t = cp.Variable(M)
    obj = cp.Minimize(cp.real(cp.sum([cp.trace(w) for w in W])))
    constraints = []
    for m in range(M):
        Q = (1+1/gamma[m])*W[m] - cp.sum(W)
        r = Q @ H_short[:,m:m+1]
        s = H_short[:,m:m+1].conj().T @ Q @ H_short[:,m:m+1] - sigma_sq[m:m+1];
        Z = cp.hstack((Q+t[m]*np.eye(N), r))
        Z = cp.vstack((Z, cp.hstack((cp.conj(cp.transpose(r)), s-t[m:m+1]*epsi[m:m+1]**2 ))))
        constraints += [W[m] >> 0]
        constraints += [Z >> 0]
    constraints += [t >= 0]
    
    prob = cp.Problem(obj, constraints)
    prob.solve(verbose=True)

    w_sols = []
    for w in W:
        evec, rank1 = helper.get_rank1(w.value)
        assert rank1, 'solution not rank 1'
        w_sols.append(np.reshape(evec, (N,1))) 

    W_sol = np.concatenate(w_sols, axis=1)

    # return order: w_solution, objective_value, optimality

    if prob.status in ['infeasible', 'unbounded']:
        # print('infeasible antenna solution')
        return np.zeros((N,M)), np.inf, False

    return W_sol, prob.objective.value, True


# Prepare Data

N=12
M=8

sigma_sq= 0.1*np.ones(M)
gamma= (10**(6/10))*np.ones(M) #SINR levels, from -10dB to 20dB
epsi= np.sqrt(0.1)*np.ones(M)

H = (np.random.randn(N,M) + 1j*np.random.randn(N,M))/np.sqrt(2);

# H =np.array([[ 0.1414 + 0.3571j,   0.3104 + 0.8294j,  -0.2649 + 0.7797j],
#     [-0.7342 + 0.0286j,  -0.0290 - 0.1337j,  -1.1732 - 0.2304j],
#     [ 0.3743 - 0.1801j,   0.2576 + 0.0612j,  -0.2194 - 0.4121j],
#     [-1.5677 - 0.3873j,  -0.5400 + 1.0695j,  -0.4384 + 0.5315j],
#     [ 0.0760 + 1.1855j,   0.3886 + 1.2680j,  -0.7245 - 0.6108j]])

z_mask = np.random.binomial(size=N, n=1, p= 0.1)
z_sol = np.random.binomial(size=N, n=1, p= 0.1)

num_offs = np.sum(z_mask*(1-z_sol))

import time

t1 = time.time()
w, obj, optimal = solve_rsdr_with_selected_antennas(H, gamma, sigma_sq, epsi, z_mask, z_sol)
time_taken = time.time()-t1

print(w)
print(obj)
print(optimal)
print('time taken', time_taken)

# W = []
# for i in range(M):
#     W.append(cp.Variable((N,N), hermitian=True))

# t = cp.Variable(M)

# obj = cp.Minimize(cp.real(cp.sum([cp.trace(w) for w in W])))


# constraints = []
# for m in range(M):
#     Q = (1+1/gamma[m])*W[m] - cp.sum(W)
#     r = Q @ H[:,m:m+1]
#     s = H[:,m:m+1].conj().T @ Q @ H[:,m:m+1] - sigma_sq[m:m+1];
#     Z = cp.hstack((Q+t[m]*np.eye(N), r))
#     Z = cp.vstack((Z, cp.hstack((cp.conj(cp.transpose(r)), s-t[m:m+1]*epsi[m:m+1]**2 ))))
# #     constraints += [Z >= 0]
#     constraints += [W[m] >> 0]
#     constraints += [Z >> 0]

# constraints += [t >= 0]

# prob = cp.Problem(obj, constraints)
# prob.solve()



[0 0 0 0 0 0 1 0 0 1 0 0] [0 0 0 0 0 1 0 0 0 0 0 0]
0 0
0 0
0 1
pruning 9
0 0
0 0
0 1
pruning 6
1 0
0 0
0 0
0 0
0 0
0 0
12 8
h short (10, 8)
                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Mar 31 03:37:45 PM: Your problem has 808 variables, 17 constraints, and 0 parameters.
(CVXPY) Mar 31 03:37:45 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 31 03:37:45 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 31 03:37:45 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 31 03:37:

/scratch/sagar/Projects/combopt/venv/lib/python3.6/site-packages/cvxpy/expressions/expression.py:556: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)
/scratch/sagar/Projects/combopt/venv/lib/python3.6/site-packages/cvxpy/expressions/expression.py:556: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 2 times so far.

  warnings

(CVXPY) Mar 31 03:37:45 PM: Applying reduction CvxAttr2Constr
(CVXPY) Mar 31 03:37:45 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Mar 31 03:37:47 PM: Applying reduction MOSEK
(CVXPY) Mar 31 03:37:50 PM: Finished problem compilation (took 4.699e+00 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Mar 31 03:37:50 PM: Invoking solver MOSEK  to obtain a solution.


Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1248            
  Cones                  : 0               
  Scalar variables       : 8               
  Matrix variables       : 16              
  Integer variables      : 0               

Optimizer started.
Problem
  Name         

In [61]:
z_mask

array([0, 0, 0, 0, 0, 0])

In [54]:
z_sol

array([1, 0, 1, 0, 0, 0])

In [28]:
import models.helper as helper

def solve_rsdr_with_selected_antennas(H, gamma, sigma_sq, epsi, z_mask, z_sol):
    """
    off_ants: antenna indices that need to be turned off
    """
    num_off_ants = np.sum(z_mask*(1-z_sol))

    N, M = H.shape
    print(H.shape)
    assert num_off_ants<N, 'number of allowed antennas < 1'
    N = N - num_off_ants

    H_short = H.copy()

    for n in range(N-1, -1, -1):
        if z_mask[n] and not z_sol[n]:
            H_short = np.concatenate((H_short[:n, :], H_short[n+1:, :]), axis=0)
    print('h short', H_short.shape)
    W = []
    for i in range(M):
        W.append(cp.Variable((N,N), hermitian=True))

    t = cp.Variable(M)

    obj = cp.Minimize(cp.real(cp.sum([cp.trace(w) for w in W])))


    constraints = []
    for m in range(M):
        Q = (1+1/gamma[m])*W[m] - cp.sum(W)
        r = Q @ H_short[:,m:m+1]
        s = H_short[:,m:m+1].conj().T @ Q @ H[:,m:m+1] - sigma_sq[m:m+1];
        Z = cp.hstack((Q+t[m]*np.eye(N), r))
        Z = cp.vstack((Z, cp.hstack((cp.conj(cp.transpose(r)), s-t[m:m+1]*epsi[m:m+1]**2 ))))
        constraints += [W[m] >> 0]
        constraints += [Z >> 0]

    constraints += [t >= 0]

    prob = cp.Problem(obj, constraints)
    prob.solve()

    w_sols = []
    for w in W:
        evec, rank1 = helper.get_rank1(w.value)
        assert rank1, 'solution not rank 1'
        w_sols.append(np.reshape(evec, (N,1))) 

    W_sol = np.concatenate(w_sols, axis=1)

    # return order: w_solution, objective_value, optimality

    if prob.status in ['infeasible', 'unbounded']:
        # print('infeasible antenna solution')
        return np.zeros((N,M)), np.inf, False

    return W_sol, prob.objective.value, True


In [19]:
np.argnp.linalg.eig(np.random.rand(5,5))[1].shape

(5, 5)

In [23]:
np.argmax(np.random.rand(5))

4

In [16]:
evec

NameError: name 'evec' is not defined

In [14]:
np.concatenate((np.random.rand(5,3), np.random.rand(2,3)), axis=0).shape

(7, 3)

In [12]:
list(range(5,1,-1))

[5, 4, 3, 2]

In [9]:
off_ants = np.random.permutation(5)
off_ants = off_ants[]

In [10]:
off_ants

array([0, 4, 1, 2, 3])

In [5]:
H =np.array([[ 0.1414 + 0.3571j,   0.3104 + 0.8294j,  -0.2649 + 0.7797j],
    [-0.7342 + 0.0286j,  -0.0290 - 0.1337j,  -1.1732 - 0.2304j],
    [ 0.3743 - 0.1801j,   0.2576 + 0.0612j,  -0.2194 - 0.4121j],
    [-1.5677 - 0.3873j,  -0.5400 + 1.0695j,  -0.4384 + 0.5315j],
    [ 0.0760 + 1.1855j,   0.3886 + 1.2680j,  -0.7245 - 0.6108j]])

In [6]:
H.shape

(5, 3)

In [73]:
prob.objective.value

0.8949739357775427

In [70]:
s

Expression(AFFINE, UNKNOWN, (1, 1))

In [65]:
a  = cp.Expression()
a = cp.reshape(s[m]-t[m:m+1]*epsi[m:m+1]**2, (1,1))

IndexError: Index 1 is out of bounds for axis 0 with size 1.

In [71]:
s-t[m:m+1]*epsi[m:m+1]**2

/scratch/sagar/Projects/combopt/venv/lib/python3.6/site-packages/cvxpy/expressions/expression.py:556: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 21 times so far.

  warnings.warn(msg, UserWarning)


Expression(AFFINE, UNKNOWN, (1, 1))

In [62]:
a

Expression(AFFINE, UNKNOWN, (1,))

In [54]:
t[m:m+1]

Expression(AFFINE, UNKNOWN, (1,))

In [52]:
s.shape

(1, 1)

In [46]:
s[m]-t[m]*epsi[m]**2

IndexError: Too many indices for expression.

In [48]:
s

Expression(AFFINE, UNKNOWN, ())

In [47]:
t

Variable((8,))

In [63]:
cp.conj(cp.transpose(r))

Expression(AFFINE, UNKNOWN, (1, 12))

In [45]:
cp.hstack((cp.conj(cp.transpose(r)), ))

ValueError: All the input dimensions except for axis 1 must match exactly.

In [40]:
cp.transpose(H)

Expression(CONSTANT, UNKNOWN, (8, 12))

In [33]:
r.conj()

AttributeError: 'MulExpression' object has no attribute 'conj'

In [26]:
H[:,m:m+1]

array([[ 0.31322818+0.46717518j],
       [-1.01539291+1.23774352j],
       [-0.15562012+0.79618297j],
       [ 0.87984116+0.33675527j],
       [-0.33312421-0.17398508j],
       [-0.58205607+0.8493666j ],
       [ 0.78445659+0.83693066j],
       [ 0.7357013 +0.04553042j],
       [-0.50804939+1.32426427j],
       [-0.21028306-0.89154786j],
       [-0.18899709-0.01431549j],
       [-0.19061622-0.37351394j]])

In [39]:
N,M = 5,2
max_ant = 3
W_sol = np.random.randn(N,M)
z_mask = np.random.binomial(n=1, p=.5, size=N)
z_sol = np.random.binomial(n=1, p=.5, size=N)

power_w = np.linalg.norm(W_sol, axis=1)
power_w = (1-z_mask)*power_w
used_ant = np.sum(z_mask*z_sol)
assert used_ant <= max_ant, 'used antennas already larger than max allowed antennas'
if used_ant == max_ant:
    print('here')
#     return z_mask*z_sol

z_feas = z_mask*z_sol
z_feas[np.flip(np.argsort(power_w))[:max_ant-used_ant]] = 1 
print(z_mask)
print(z_sol)

print(power_w)
print(z_feas)

here
[0 0 1 1 1]
[0 0 1 1 1]
[0.17835332 1.8514348  0.         0.         0.        ]
[0 0 1 1 1]


In [33]:
np.argsort(power_w)

array([4, 1, 0, 2, 3])

In [1]:
import numpy as np

a = np.random.rand(5)

In [2]:
np.random.permutation(a)

array([0.98426867, 0.34367917, 0.28750968, 0.57140194, 0.32079667])

In [10]:
import numpy as np

instances = np.random.rand(5,2,3)
print(instances)

i = 0
instances = np.concatenate((instances[:i,::], instances[i+1:,::]), axis=0)

[[[0.59435773 0.5134633  0.52743774]
  [0.63610901 0.09154458 0.46513214]]

 [[0.40219084 0.88039297 0.17470154]
  [0.44435442 0.24612651 0.17332009]]

 [[0.80876549 0.47262972 0.47260315]
  [0.09748929 0.87656048 0.20163077]]

 [[0.32735498 0.4093701  0.30735459]
  [0.80742646 0.58730433 0.51954617]]

 [[0.55269942 0.42734868 0.46422979]
  [0.34728958 0.54230804 0.65788464]]]


In [11]:
len(instances)

4

In [16]:
class CustomException(Exception):
    pass

try:
    raise CustomException('an error occured')
except CustomException as e:
    print('message', e)
    
    
print(5)

message an error occured
5


In [ ]:
 matrix [[-1.44133888+0.35172095j  0.75348816+0.00645361j  1.35778683+0.64247147j
   0.83363469-1.24131836j]
 [-0.01661496-1.02176565j  0.89624182-0.29839284j  1.10508271-0.81603941j
   0.55242791-0.56316898j]
 [ 0.04253986+0.55146412j -1.18925535+0.43070676j -0.5764893 +0.2167988j
   0.30590856+0.1940031j ]
 [-1.43286293+1.75314882j -0.15042443+0.66050685j  0.22175695-0.15327095j
  -0.71797835-0.06075578j]
 [ 0.28747101-0.16796463j  0.22154135+0.20050112j  1.482013  +0.85440646j
   1.30426042-0.45960364j]
 [ 0.41682604-0.41250976j  0.12485053-0.0746889j  -0.87193747-1.89902827j
   0.12141707+0.34111201j]
 [ 1.22917077+0.961712j    1.48352611+0.66406165j  1.66216921+0.7029772j
   1.95702491-0.74891918j]
 [-0.7671263 -0.28411103j  0.24981818+0.44662971j -0.65997513+0.47891181j
  -0.72319534+0.29522664j]] 
z_mask [1. 1. 1. 1. 1. 1. 1. 1.] 
 z_sol [0. 1. 1. 0. 0. 0. 0. 1.]
